In [2]:
import pandas as pd
import geopandas as gpd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Reading file
eaz = gpd.read_file('atx_eaz_multi_year.geojson')

# Convert to GeoJSON for Plotly
geojson = eaz.__geo_interface__

In [3]:
# Variables that will be visualized
variables = ['indexed_vulnerability', 'pct_poc', 'pct_underserved_poc', 'median_hh_inc', 'pct_with_disability', 'pct_food_stamps', 'pct_rent_over35', 'pct_wo_broadband', 'pct_less_vehicles', 'pct_over_70']

In [4]:
# Reshaping the dataframe
eaz_wide = eaz.pivot_table(index='GEOID', columns='acs_year', values=variables)

# Renaming the columns, since currently multiindex
eaz_wide.columns = [f'{var}_{year}' for var, year in eaz_wide.columns]

# Reset index to make 'GEOID' a column again
eaz_wide.reset_index(inplace=True)

In [5]:
eaz_wide

,GEOID,indexed_vulnerability_2019,indexed_vulnerability_2020,indexed_vulnerability_2021,indexed_vulnerability_2022,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021,median_hh_inc_2022,pct_food_stamps_2019,...,pct_underserved_poc_2021,pct_underserved_poc_2022,pct_with_disability_2019,pct_with_disability_2020,pct_with_disability_2021,pct_with_disability_2022,pct_wo_broadband_2019,pct_wo_broadband_2020,pct_wo_broadband_2021,pct_wo_broadband_2022
0,48021950101,57.295374,46.190704,46.452969,49.173554,-76424.0,-85345.0,-90498.0,-97707.0,0.065128,...,0.334755,0.381440,1.000000,1.000000,1.000000,1.000000,0.251582,0.225170,0.237634,0.190802
1,48021950102,57.295374,47.580259,37.624803,43.336777,-76424.0,-88097.0,-92207.0,-90570.0,0.065128,...,0.320066,0.353996,1.000000,1.000000,1.000000,1.000000,0.251582,0.120954,0.093988,0.113910
2,48021950201,74.631418,87.685673,69.574356,67.407025,-54554.0,-54910.0,-60602.0,-63393.0,0.223297,...,0.745209,0.691814,0.987411,1.000000,1.000000,0.997251,0.192261,0.229972,0.086614,0.180992
3,48021950202,74.631418,62.529947,54.072517,59.400826,-54554.0,-74537.0,-95333.0,-87979.0,0.223297,...,0.600648,0.618952,0.987411,0.981757,0.983112,0.980175,0.192261,0.099307,0.081229,0.085744
4,48021950301,41.992883,42.117873,44.508671,39.617769,-71765.0,-96800.0,-112775.0,-109141.0,0.123758,...,0.373107,0.300779,0.995821,0.986371,0.989726,0.986229,0.115018,0.105447,0.073067,0.089537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,48491021517,58.820539,61.379971,52.863899,63.429752,-89947.0,-81078.0,-86750.0,-74699.0,0.094364,...,0.461916,0.492554,1.000000,1.000000,1.000000,1.000000,0.003495,0.000000,0.000000,0.000000
499,48491021518,58.820539,62.577863,63.899107,67.252066,-89947.0,-101929.0,-97404.0,-114783.0,0.094364,...,0.470038,0.557731,1.000000,1.000000,1.000000,1.000000,0.003495,0.007812,0.011152,0.000000
500,48491021601,37.214032,48.155247,47.398844,55.010331,-71795.0,-72610.0,-82620.0,-83025.0,0.063924,...,0.428906,0.482050,0.993718,0.997904,0.995206,0.996011,0.093671,0.083779,0.071958,0.062218
501,48491021602,52.770717,58.025874,53.441934,19.576446,-71535.0,-73000.0,-82691.0,-113011.0,0.113111,...,0.306588,0.192342,1.000000,1.000000,1.000000,0.989231,0.186375,0.153203,0.124383,0.063187


In [6]:
# Getting information on the first and last year
min_year = eaz['acs_year'].min()
max_year = eaz['acs_year'].max()

In [7]:
# Creating difference since 2019 columns
for var in variables:
    for year in range(min_year, max_year + 1):
        base_var = f'{var}_{min_year}'
        compare_var = f'{var}_{year}'

        diff_var = f'difference_since_2019_{year}_{var}'

        eaz_wide[diff_var] = eaz_wide[compare_var] - eaz_wide[base_var]
    

In [8]:
eaz_wide.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 81 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   GEOID                                             503 non-null    object 
 1   indexed_vulnerability_2019                        502 non-null    float64
 2   indexed_vulnerability_2020                        502 non-null    float64
 3   indexed_vulnerability_2021                        502 non-null    float64
 4   indexed_vulnerability_2022                        502 non-null    float64
 5   median_hh_inc_2019                                503 non-null    float64
 6   median_hh_inc_2020                                503 non-null    float64
 7   median_hh_inc_2021                                503 non-null    float64
 8   median_hh_inc_2022                                503 non-null    float64
 9   pct_food_stamps_2019 

In [9]:
# Keeping columns that start with 'GEOID' or 'difference_since'
columns_to_keep = eaz_wide.filter(regex='^(GEOID|difference_since)').columns
eaz_wide_filtered = eaz_wide[columns_to_keep]

In [10]:
df_long = eaz_wide_filtered.melt(id_vars=['GEOID'], var_name='variable', value_name='value')

In [11]:
df_long.head()

,GEOID,variable,value
0,48021950101,difference_since_2019_2019_indexed_vulnerability,0.0
1,48021950102,difference_since_2019_2019_indexed_vulnerability,0.0
2,48021950201,difference_since_2019_2019_indexed_vulnerability,0.0
3,48021950202,difference_since_2019_2019_indexed_vulnerability,0.0
4,48021950301,difference_since_2019_2019_indexed_vulnerability,0.0


In [12]:
# Extract the correct year and base variable
df_long[['ignore', 'acs_year', 'base_var']] = df_long['variable'].str.extract(r'(difference_since_2019)_(\d{4})_(.*)')

# Recreate variable name with the proper prefix
df_long['base_var'] = 'difference_since_2019_' + df_long['base_var']

In [13]:
df_long

,GEOID,variable,value,ignore,acs_year,base_var
0,48021950101,difference_since_2019_2019_indexed_vulnerability,0.000000,difference_since_2019,2019,difference_since_2019_indexed_vulnerability
1,48021950102,difference_since_2019_2019_indexed_vulnerability,0.000000,difference_since_2019,2019,difference_since_2019_indexed_vulnerability
2,48021950201,difference_since_2019_2019_indexed_vulnerability,0.000000,difference_since_2019,2019,difference_since_2019_indexed_vulnerability
3,48021950202,difference_since_2019_2019_indexed_vulnerability,0.000000,difference_since_2019,2019,difference_since_2019_indexed_vulnerability
4,48021950301,difference_since_2019_2019_indexed_vulnerability,0.000000,difference_since_2019,2019,difference_since_2019_indexed_vulnerability
...,...,...,...,...,...,...
20115,48491021517,difference_since_2019_2022_pct_over_70,0.043179,difference_since_2019,2022,difference_since_2019_pct_over_70
20116,48491021518,difference_since_2019_2022_pct_over_70,0.007439,difference_since_2019,2022,difference_since_2019_pct_over_70
20117,48491021601,difference_since_2019_2022_pct_over_70,0.002791,difference_since_2019,2022,difference_since_2019_pct_over_70
20118,48491021602,difference_since_2019_2022_pct_over_70,-0.014701,difference_since_2019,2022,difference_since_2019_pct_over_70


In [14]:
# Filter out the rows we are interested in
df_filtered = df_long[df_long['base_var'].isin(['difference_since_2019_' + var for var in variables])]

In [15]:
df_filtered

,GEOID,variable,value,ignore,acs_year,base_var
0,48021950101,difference_since_2019_2019_indexed_vulnerability,0.000000,difference_since_2019,2019,difference_since_2019_indexed_vulnerability
1,48021950102,difference_since_2019_2019_indexed_vulnerability,0.000000,difference_since_2019,2019,difference_since_2019_indexed_vulnerability
2,48021950201,difference_since_2019_2019_indexed_vulnerability,0.000000,difference_since_2019,2019,difference_since_2019_indexed_vulnerability
3,48021950202,difference_since_2019_2019_indexed_vulnerability,0.000000,difference_since_2019,2019,difference_since_2019_indexed_vulnerability
4,48021950301,difference_since_2019_2019_indexed_vulnerability,0.000000,difference_since_2019,2019,difference_since_2019_indexed_vulnerability
...,...,...,...,...,...,...
20115,48491021517,difference_since_2019_2022_pct_over_70,0.043179,difference_since_2019,2022,difference_since_2019_pct_over_70
20116,48491021518,difference_since_2019_2022_pct_over_70,0.007439,difference_since_2019,2022,difference_since_2019_pct_over_70
20117,48491021601,difference_since_2019_2022_pct_over_70,0.002791,difference_since_2019,2022,difference_since_2019_pct_over_70
20118,48491021602,difference_since_2019_2022_pct_over_70,-0.014701,difference_since_2019,2022,difference_since_2019_pct_over_70


In [16]:
# Pivot the data to have separate columns for each variable
df_pivot = df_filtered.pivot_table(
    index=['GEOID', 'acs_year'], 
    columns='base_var', 
    values='value'
).reset_index()

In [17]:
df_pivot

base_var,GEOID,acs_year,difference_since_2019_indexed_vulnerability,difference_since_2019_median_hh_inc,difference_since_2019_pct_food_stamps,difference_since_2019_pct_less_vehicles,difference_since_2019_pct_over_70,difference_since_2019_pct_poc,difference_since_2019_pct_rent_over35,difference_since_2019_pct_underserved_poc,difference_since_2019_pct_with_disability,difference_since_2019_pct_wo_broadband
0,48021950101,2019,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,48021950101,2020,-11.104669,-8921.0,0.011368,-0.036250,-0.030880,0.016289,-0.092887,-0.065747,0.000000,-0.026412
2,48021950101,2021,-10.842405,-14074.0,-0.002405,-0.037123,-0.036298,0.080421,-0.036853,-0.011593,0.000000,-0.013948
3,48021950101,2022,-8.121820,-21283.0,-0.018162,-0.033117,-0.018647,0.108314,-0.032824,0.035092,0.000000,-0.060779
4,48021950102,2019,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2007,48491021602,2022,-33.194271,-41476.0,-0.019704,0.022996,-0.014701,-0.100937,-0.200000,-0.119805,-0.010769,-0.123189
2008,48491021603,2019,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2009,48491021603,2020,2.454149,-3177.0,-0.010038,0.013618,0.021000,-0.023037,-0.049725,-0.021419,0.000815,0.011585
2010,48491021603,2021,5.427414,-3427.0,0.011695,-0.015532,0.024776,0.015739,-0.078785,0.010198,0.001314,-0.023074


In [18]:
df_sorted = df_pivot.sort_values(by=['acs_year', 'GEOID'], ascending=[True, True])

In [19]:
df_sorted

base_var,GEOID,acs_year,difference_since_2019_indexed_vulnerability,difference_since_2019_median_hh_inc,difference_since_2019_pct_food_stamps,difference_since_2019_pct_less_vehicles,difference_since_2019_pct_over_70,difference_since_2019_pct_poc,difference_since_2019_pct_rent_over35,difference_since_2019_pct_underserved_poc,difference_since_2019_pct_with_disability,difference_since_2019_pct_wo_broadband
0,48021950101,2019,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,48021950102,2019,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,48021950201,2019,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
12,48021950202,2019,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
16,48021950301,2019,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,48491021517,2022,4.609213,15248.0,-0.053209,-0.046551,0.043179,-0.113939,0.009257,-0.070276,0.000000,-0.003495
1999,48491021518,2022,8.431527,-24836.0,-0.007882,0.020918,0.007439,-0.006260,0.165055,-0.005099,0.000000,-0.003495
2003,48491021601,2022,17.796299,-11230.0,-0.015232,0.045074,0.002791,0.104593,0.110642,0.069996,0.002293,-0.031453
2007,48491021602,2022,-33.194271,-41476.0,-0.019704,0.022996,-0.014701,-0.100937,-0.200000,-0.119805,-0.010769,-0.123189


In [20]:
eaz

,GEOID,2019_tract,NAME,indexed_vulnerability,eaz_type,pct_poc,pct_underserved_poc,median_hh_inc,pct_food_stamps,pct_rent_over35,pct_wo_broadband,pct_less_vehicles,pct_with_disability,pct_over_70,acs_year,difference_since_2019,geometry
0,48453002419,48453002419,"Census Tract 24.19, Travis County, Texas",86.578546,Most Vulnerable,0.876036,0.824469,-40318.0,0.132466,0.390693,0.188343,0.269030,1.000000,0.007280,2019,NaN,"POLYGON ((-97.77052 30.19054, -97.76986 30.191..."
1,48453002422,48453002422,"Census Tract 24.22, Travis County, Texas",58.210473,Medium Vulnerable,0.550741,0.486820,-60728.0,0.026549,0.336053,0.074484,0.103594,1.000000,0.053377,2019,NaN,"POLYGON ((-97.79891 30.17448, -97.79888 30.174..."
2,48453002423,48453002423,"Census Tract 24.23, Travis County, Texas",54.753432,Medium Vulnerable,0.472782,0.431663,-69654.0,0.038712,0.512406,0.051361,0.130224,1.000000,0.041785,2019,NaN,"POLYGON ((-97.82000 30.17389, -97.81865 30.176..."
3,48453002424,48453002424,"Census Tract 24.24, Travis County, Texas",56.227758,Medium Vulnerable,0.524371,0.515393,-56667.0,0.092867,0.378890,0.088829,0.103592,0.989225,0.072858,2019,NaN,"POLYGON ((-97.81278 30.18564, -97.81274 30.185..."
4,48453002430,48453002430,"Census Tract 24.30, Travis County, Texas",88.103711,Most Vulnerable,0.798689,0.779020,-53882.0,0.241997,0.239044,0.224072,0.263235,1.000000,0.017354,2019,NaN,"POLYGON ((-97.76980 30.17767, -97.76959 30.177..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2007,48491021517,None,Census Tract 215.17; Williamson County; Texas,63.429752,Medium-High Vulnerable,0.534139,0.492554,-74699.0,0.041155,0.321622,0.000000,0.011794,1.000000,0.062096,2022,4.609213,"POLYGON ((-97.66789 30.53036, -97.66452 30.531..."
2008,48491021518,None,Census Tract 215.18; Williamson County; Texas,67.252066,Medium-High Vulnerable,0.641817,0.557731,-114783.0,0.086483,0.477419,0.000000,0.079263,1.000000,0.026355,2022,8.431527,"POLYGON ((-97.66838 30.53890, -97.66741 30.539..."
2009,48491021601,None,Census Tract 216.01; Williamson County; Texas,55.010331,Medium Vulnerable,0.553749,0.482050,-83025.0,0.048693,0.377309,0.062218,0.070537,0.996011,0.049708,2022,17.796299,"POLYGON ((-97.64400 30.72371, -97.64286 30.727..."
2010,48491021602,None,Census Tract 216.02; Williamson County; Texas,19.576446,Least Vulnerable,0.230631,0.192342,-113011.0,0.093407,0.000000,0.063187,0.057650,0.989231,0.087347,2022,-33.194271,"POLYGON ((-97.65017 30.70466, -97.64994 30.705..."


In [21]:
# Merging the two dataframes together
eaz_filtered = eaz.drop(columns=['difference_since_2019'], inplace=False)

In [22]:
df_sorted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2012 entries, 0 to 2011
Data columns (total 12 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   GEOID                                        2012 non-null   object 
 1   acs_year                                     2012 non-null   object 
 2   difference_since_2019_indexed_vulnerability  2008 non-null   float64
 3   difference_since_2019_median_hh_inc          2012 non-null   float64
 4   difference_since_2019_pct_food_stamps        2012 non-null   float64
 5   difference_since_2019_pct_less_vehicles      2012 non-null   float64
 6   difference_since_2019_pct_over_70            2008 non-null   float64
 7   difference_since_2019_pct_poc                2008 non-null   float64
 8   difference_since_2019_pct_rent_over35        2012 non-null   float64
 9   difference_since_2019_pct_underserved_poc    2008 non-null   float64
 10  diffe

In [23]:
df_sorted["acs_year"] = pd.to_numeric(df_sorted["acs_year"])
merged_data= eaz_filtered.merge(df_sorted, on=["GEOID","acs_year"])

In [24]:
merged_data

,GEOID,2019_tract,NAME,indexed_vulnerability,eaz_type,pct_poc,pct_underserved_poc,median_hh_inc,pct_food_stamps,pct_rent_over35,...,difference_since_2019_indexed_vulnerability,difference_since_2019_median_hh_inc,difference_since_2019_pct_food_stamps,difference_since_2019_pct_less_vehicles,difference_since_2019_pct_over_70,difference_since_2019_pct_poc,difference_since_2019_pct_rent_over35,difference_since_2019_pct_underserved_poc,difference_since_2019_pct_with_disability,difference_since_2019_pct_wo_broadband
0,48453002419,48453002419,"Census Tract 24.19, Travis County, Texas",86.578546,Most Vulnerable,0.876036,0.824469,-40318.0,0.132466,0.390693,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,48453002422,48453002422,"Census Tract 24.22, Travis County, Texas",58.210473,Medium Vulnerable,0.550741,0.486820,-60728.0,0.026549,0.336053,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,48453002423,48453002423,"Census Tract 24.23, Travis County, Texas",54.753432,Medium Vulnerable,0.472782,0.431663,-69654.0,0.038712,0.512406,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,48453002424,48453002424,"Census Tract 24.24, Travis County, Texas",56.227758,Medium Vulnerable,0.524371,0.515393,-56667.0,0.092867,0.378890,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,48453002430,48453002430,"Census Tract 24.30, Travis County, Texas",88.103711,Most Vulnerable,0.798689,0.779020,-53882.0,0.241997,0.239044,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2007,48491021517,None,Census Tract 215.17; Williamson County; Texas,63.429752,Medium-High Vulnerable,0.534139,0.492554,-74699.0,0.041155,0.321622,...,4.609213,15248.0,-0.053209,-0.046551,0.043179,-0.113939,0.009257,-0.070276,0.000000,-0.003495
2008,48491021518,None,Census Tract 215.18; Williamson County; Texas,67.252066,Medium-High Vulnerable,0.641817,0.557731,-114783.0,0.086483,0.477419,...,8.431527,-24836.0,-0.007882,0.020918,0.007439,-0.006260,0.165055,-0.005099,0.000000,-0.003495
2009,48491021601,None,Census Tract 216.01; Williamson County; Texas,55.010331,Medium Vulnerable,0.553749,0.482050,-83025.0,0.048693,0.377309,...,17.796299,-11230.0,-0.015232,0.045074,0.002791,0.104593,0.110642,0.069996,0.002293,-0.031453
2010,48491021602,None,Census Tract 216.02; Williamson County; Texas,19.576446,Least Vulnerable,0.230631,0.192342,-113011.0,0.093407,0.000000,...,-33.194271,-41476.0,-0.019704,0.022996,-0.014701,-0.100937,-0.200000,-0.119805,-0.010769,-0.123189


In [26]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Sample dataframe (replace with your actual dataframe)
# eaz = pd.read_csv('your_data.csv') 
# geojson = 'your_geojson_file'

# Define the variables and their absolute and relative versions
variables = [
    ('indexed_vulnerability', 'difference_since_2019_indexed_vulnerability'),
    ('pct_poc', 'difference_since_2019_pct_poc'),
    ('pct_underserved_poc', 'difference_since_2019_pct_underserved_poc')
]

# Create subplots
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'choroplethmapbox'}, {'type': 'bar'}]],
    column_widths=[1.0, 0.55]
)

# Defining the order of eaz_type categories
order = ['Least Vulnerable', 'Medium-Low Vulnerable', 'Medium Vulnerable', 'Medium-High Vulnerable', 'Most Vulnerable']
order_mapping = {name: i for i, name in enumerate(order)}

# Add traces for each variable, year, and type (absolute/relative)
for var, rel_var in variables:
    for year in merged_data['acs_year'].unique():
        year_data = merged_data[merged_data['acs_year'] == year]

        for var_type, is_relative in [('Absolute', False), ('Relative', True)]:
            # Choose the variable based on the type
            data_column = rel_var if is_relative else var

            # Define hover template based on variable type
            # hovertemplate = (
            #     f"<b>%{{customdata[0]}}</b><br>" +
            #     f"<b>Year:</b> %{{customdata[1]}}<br>" +
            #     f"<b>{var}:</b> %{{z:.2f}}<br>" +
            #     (f"<b>Difference from 2019:</b> %{{customdata[2]:+.2f}}<br>" if is_relative else "") +
            #     "<extra></extra>"
            # )

            hovertemplate = (
                f"<b>%{{customdata[0]}}</b><br>" +
                f"<b>Year:</b> %{{customdata[1]}}<br>" +
                f"<b>{var}:</b> %{{z:.2f}}<br>" +
                "<extra></extra>"
            )

            # Prepare custom data for hover
            customdata = year_data[['NAME', 'acs_year']].copy()     # Changed to remove difference since 2019
            # customdata['difference_since_2019'] = customdata['difference_since_2019'].apply(
            #     lambda x: f"{x:+.2f}" if pd.notnull(x) else "N/A"
            # )

            fig.add_trace(go.Choroplethmapbox(
                geojson=geojson,
                locations=year_data.index,
                z=year_data[data_column],
                colorscale="Reds",
                zmin=merged_data[data_column].min(),
                zmax=merged_data[data_column].max(),
                marker_opacity=0.8,
                marker_line_width=0.25,
                visible=(year == merged_data['acs_year'].min() and var == 'indexed_vulnerability' and var_type == 'Absolute'),  # Only the first year and default variable visible initially
                name=f"{year}_{var}_{var_type}",  # Name includes variable and type
                hovertemplate=hovertemplate,
                customdata=customdata[['NAME', 'acs_year']],        # changed to remove difference since 2019
                colorbar=dict(
                    x=0.1,
                    y=-0.015,
                    thickness=10,
                    outlinecolor='rgba(0,0,0,0)',  # Transparent outline
                    bgcolor='rgba(255,255,255,1)', # White background
                    len=0.2,
                    orientation='h'
                )
            ), row=1, col=1)

            # Bar graph traces
            bar_data = year_data[['eaz_type']].value_counts().reset_index()
            bar_data.columns = ['eaz_type', 'count']
            bar_data['order'] = bar_data['eaz_type'].map(order_mapping)
            bar_data = bar_data.sort_values('order')  # Sort by the specified order

            fig.add_trace(go.Bar(
                x=bar_data['eaz_type'],
                y=bar_data['count'],
                marker=dict(color='red'),
                showlegend=False,
                visible=(year == merged_data['acs_year'].min() and var == 'indexed_vulnerability' and var_type == 'Absolute'),
                hovertemplate=(
                    "<b>Number of census tracts:</b> %{y}<br>" +
                    "<extra></extra>"
                )
            ), row=1, col=2)

# Update layout with slider steps
steps = []
for year in merged_data['acs_year'].unique():
    for var, rel_var in variables:
        for var_type in ['Absolute', 'Relative']:
            visible = []

            for y in merged_data['acs_year'].unique():
                for v, r in variables:
                    for vt in ['Absolute', 'Relative']:
                        visible.append(y == year and v == var and vt == var_type)
                        visible.append(y == year and v == var and vt == var_type)

            step = dict(
                label=f"{year} - {var} ({var_type})",
                method="update",
                args=[
                    {
                        "visible": visible,
                        "title": f"{var.replace('_', ' ').title()} ({var_type}) - {year}"
                    }
                ],
            )
            steps.append(step)

# Update layout with dropdown menu for variable selection
dropdown_buttons = [
    {
        "label": var.replace('_', ' ').title(),
        "method": "update",
        "args": [
            {"visible": [(v == var) for v, _ in variables for _ in range(len(merged_data['acs_year'].unique()) * 4)]},
            {"title": var.replace('_', ' ').title()}
        ]
    } for var, _ in variables
]

# Update layout with toggle buttons for absolute/relative switching
toggle_buttons = [
    {
        "label": "Absolute",
        "method": "update",
        "args": [
            {"visible": [not is_relative for _, rel_var in variables for _ in range(len(merged_data['acs_year'].unique()) * 2) for is_relative in [False, True]]},
            {"title": "Absolute Values"}
        ]
    },
    {
        "label": "Relative",
        "method": "update",
        "args": [
            {"visible": [is_relative for _, rel_var in variables for _ in range(len(merged_data['acs_year'].unique()) * 2) for is_relative in [False, True]]},
            {"title": "Relative Values"}
        ]
    }
]

fig.update_layout(
    title_text="Equity Analysis Zones<br><sup>Indexed Vulnerability Score</sup>",
    title_font=dict(size=25),
    title_x=0.5,
    mapbox_style="carto-positron",
    width=1200,
    height=800,
    mapbox_zoom=8,
    mapbox_center={"lat": 30.2672, "lon": -97.7431},
    sliders=[dict(
        active=0,
        steps=steps,
        currentvalue=dict(
            prefix="Selected year: ",
            font=dict(size=10, color="black"),
            visible=True,
            xanchor='left'
        )
    )],
    updatemenus=[
        {
            "buttons": dropdown_buttons,
            "direction": "down",
            "showactive": True,
            "x": 0.17,
            "y": 1.1
        },
        {
            "buttons": toggle_buttons,
            "direction": "down",
            "showactive": True,
            "x": 0.25,
            "y": 1.1
        }
    ]
)

# Customizing the look of the bar plot
fig.update_xaxes(
    tickmode='array',
    tickvals=[0, 1, 2, 3, 4],
    ticktext=['Least<br>Vulnerable', 'Medium-Low<br>Vulnerable', 'Medium<br>Vulnerable', 'Medium-High<br>Vulnerable', 'Most<br>Vulnerable'],
    tickfont=dict(size=9),
    tickangle=0, 
    row=1, col=2    # Apply to the bar plot
)

fig.update_yaxes(
    gridcolor='lightgray',
    row=1, col=2     # Apply to the bar plot
)

fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)'
)

# Show the figure
fig.show()


KeyError: "['difference_since_2019'] not in index"

In [ ]:
# Creating Plotly figure with subplots
# Create subplots
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'choroplethmapbox'}, {'type': 'bar'}]],
    column_widths=[1.0, 0.55]
)

# Defining the order of eaz_type categories
order = ['Least Vulnerable', 'Medium-Low Vulnerable', 'Medium Vulnerable', 'Medium-High Vulnerable', 'Most Vulnerable']
order_mapping = {name: i for i, name in enumerate(order)}

# Adding traces for the plots
for year in eaz['acs_year'].unique():
    year_data = eaz[eaz['acs_year'] == year]

    # Customdata formatted for hover information
    customdata = year_data[['NAME', 'acs_year', 'difference_since_2019']].copy()
    customdata['difference_since_2019'] = customdata['difference_since_2019'].apply(
        lambda x: f"{x:+.2f}" if pd.notnull(x) else "N/A"
    )

    # Chloropleth traces
    if year == 2019:    # Starting year
        hovertemplate = (
            "<b>%{customdata[0]}</b><br>" +
            "<b>Year:</b> %{customdata[1]}<br>" +
            "<b>Vulnerability Index:</b> %{z:.2f}<br>" +
            "<extra></extra>"
        )
    else:
        hovertemplate = (
            "<b>%{customdata[0]}</b><br>" +
            "<b>Year:</b> %{customdata[1]}<br>" +
            "<b>Vulnerability Index:</b> %{z:.2f}<br>" +
            "<b>Difference from 2019:</b> %{customdata[2]:+.2f}<br>" +
            "<extra></extra>"
        )
    
    fig.add_trace(go.Choroplethmapbox(
        geojson=geojson,
        locations=year_data.index,
        z=year_data['indexed_vulnerability'],
        # colorscale="RdYlGn_r",
        colorscale="Reds",
        zmin=eaz['indexed_vulnerability'].min(),
        zmax=eaz['indexed_vulnerability'].max(),
        marker_opacity=0.8,
        marker_line_width=0.25,
        visible=(year == eaz['acs_year'].min()),  # Only the first year visible initially
        name=str(year),  # Add name for the slider
        hovertemplate=hovertemplate,
        customdata=customdata[['NAME', 'acs_year', 'difference_since_2019']],
        colorbar=dict(
            x = 0.1,
            y = -0.015,
            thickness=10,
            outlinecolor='rgba(0,0,0,0)',  # Transparent outline
            bgcolor='rgba(255,255,255,1)', # White background
            len = 0.2,
            orientation = 'h'
        )
    ), row=1, col=1)


    # Bar graph traces
    bar_data = year_data[['eaz_type']].value_counts().reset_index()
    bar_data.columns = ['eaz_type', 'count']
    bar_data['order'] = bar_data['eaz_type'].map(order_mapping)
    bar_data = bar_data.sort_values('order')  # Sort by the specified order

    fig.add_trace(go.Bar(
        x=bar_data['eaz_type'],
        y=bar_data['count'],
        marker=dict(color='red'),
        showlegend=False,
        visible=(year == eaz['acs_year'].min()),
        hovertemplate=(
            "<b>Number of census tracts:</b> %{y}<br>" +
            "<extra></extra>"
        )
    ), row=1, col=2)


# Update layout
steps = []
for year in eaz['acs_year'].unique():

    visible = []

    for y in eaz['acs_year'].unique():
        visible.append(y == year)
        visible.append(y == year)

    step = dict(
        label=str(year),
        method="update",
        args=[
            {
                "visible": visible, 
                "title": f"Indexed Vulnerability - {year}"
            }
        ],
    )
    steps.append(step)

fig.update_layout(
    title_text="Equity Analysis Zones<br><sup>Indexed Vulnerability Score</sup>",
    title_font=dict(size=25),
    title_x = 0.5,
    mapbox_style="carto-positron",
    width=1200,
    height=800,
    mapbox_zoom=8,
    mapbox_center={"lat": 30.2672, "lon": -97.7431},
    sliders=[dict(
        active=0,
        steps=steps,
        currentvalue=dict(
            prefix="Selected year: ",
            font=dict(size=10, color="black"),
            visible=True,
            xanchor='left'
        )
    )]
)


# Customizing the look of the bar plot
fig.update_xaxes(
    tickmode='array',
    tickvals=[0, 1, 2, 3, 4],
    ticktext=['Least<br>Vulnerable', 'Medium-Low<br>Vulnerable', 'Medium<br>Vulnerable', 'Medium-High<br>Vulnerable', 'Most<br>Vulnerable'],
    tickfont=dict(size=9),
    tickangle=0, 
    row=1, col=2    # Apply to the bar plot
)

fig.update_yaxes(
    gridcolor='lightgray',
    # tickvals=[20, 40, 60, 80, 100, 120, 140, 160],
    row=1, col=2     # Apply to the bar plot
)

fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)'
)

# Show the figure
fig.show()

In [3]:
# Exporting to HTML
fig.write_html("eaz_map.html")